In [47]:
import csv
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
from scipy.spatial.distance import pdist, squareform
from openai import OpenAI
import json
import time
from collections import Counter
import random
import math
import re
import pandas as pd
from scipy.optimize import linear_sum_assignment
import ast
import os
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Optional
import builtins
from lshashpy3 import LSHash
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

In [48]:
client = OpenAI(
    api_key="your API key" 
)

pre_prompt = ("Please classify the following records into a two-dimensional list. Each element of the array "
              "should be a group, containing the record IDs of that group (e.g., 1, 2, 3, etc.). Ensure that "
              "each record ID is classified exactly once and appear once in the 2D array, without any "
              "duplication or omission.The output should be a two-dimensional list with no additional information!\n")

In [49]:
import csv
def get_ground_truth(file_path):
    class UnionFind:
            def __init__(self):
                self.parent = {}

            def find(self, x):
                if self.parent[x] != x:
                    self.parent[x] = self.find(self.parent[x])
                return self.parent[x]

            def union(self, x, y):
                rootX = self.find(x)
                rootY = self.find(y)
                if rootX != rootY:
                    self.parent[rootY] = rootX

            def add(self, x):
                if x not in self.parent:
                    self.parent[x] = x

    def merge_coordinates(coordinates):
            uf = UnionFind()
            ids = set()

            for ltable_id, rtable_id in coordinates:
                uf.add(ltable_id)
                uf.add(rtable_id)
                uf.union(ltable_id, rtable_id)
                ids.add(ltable_id)
                ids.add(rtable_id)

            entity_groups = {}
            for _id in ids:
                root = uf.find(_id)
                if root not in entity_groups:
                    entity_groups[root] = []
                entity_groups[root].append(_id)

            result_1 = []
            for root, records in entity_groups.items():
                result_1.append(records)

            return result_1
    data = []
    with open(file_path, newline='', encoding='MacRoman') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)  
        for row in reader:
            data.append(row)

    ground_truth_1 = merge_coordinates(data)
    print(len(ground_truth_1))
    ground_truth_new = []
    for row in ground_truth_1:
        tmp = []
        for ids in row:
            tmp.append(int(ids))
        ground_truth_new.append(tmp)
    return ground_truth_new


In [50]:
def get_data(id_list, file_path):
    lines = []
    with open(file_path, 'r', encoding='MacRoman') as file:
        reader = csv.DictReader(file)
        rows = list(reader)
        for r_id in id_list:
            for row in rows:
                if row['ID'] == str(r_id): 
                    lines.append(','.join([str(row[key]) for key in reader.fieldnames if key != 'ID']))
                    break
    prompts = '\n'.join(lines)
    return prompts

def vectorize_data(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(text.split('\n'))  
    return embeddings

def elbow_method(embeddings, max_k=5):
    if embeddings is None or embeddings.shape[0] < 2:
        return 1  
    
    distortions = []
    K = range(1, min(max_k, embeddings.shape[0]) + 1)
    
    for k in K:
        kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
        kmeans.fit(embeddings)
        distortions.append(kmeans.inertia_)
    optimal_k_index = np.argmin(distortions[1:]) + 1  
    optimal_k = K[optimal_k_index]

    print(f"best is : {optimal_k}")
    return optimal_k

def kmeans_clustering(embeddings, n_clusters):
    if embeddings is None or len(embeddings) < n_clusters:
        return np.zeros(len(embeddings), dtype=int)
    
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42,n_init=10)
    kmeans.fit(embeddings)
    return kmeans.labels_

def format_output(id_list, labels):
    clusters = {}
    for i, label in enumerate(labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(id_list[i])
    
    sorted_clusters = sorted(clusters.items(), key=lambda x: len(x[1]), reverse=True)
    
    output = []
    for cluster in sorted_clusters:
        output.append([int(item) for item in cluster[1]])
    
    return output

def read_csv_to_2d_array(file_path):
    with open(file_path, 'r', encoding='MacRoman') as file:
        reader = csv.reader(file)
        data = list(reader)
    return data

def get_prompt_from_ids(id_list, file_path):
    lines = []
    with open(file_path, 'r', encoding='MacRoman') as file:
        reader = csv.DictReader(file)
        rows = list(reader)
        for r_id in id_list:
            for row in rows:
                if row['ID'] == str(r_id): 
                    rec_str = f"Record {r_id}: "
                    rec_str += ','.join([str(row[key]) for key in reader.fieldnames if key != 'ID'])
                    lines.append(rec_str)
                    break
    return '\n'.join(lines)



In [51]:
def dynamic_sampling(original_array):
    result = []
    row_indices = {i: row.copy() for i, row in enumerate(original_array)}
    total_ids = sum(len(row) for row in original_array)
    
    while True:
        group = []
        while len(group) < 10:
            flag = False
            for i in range(len(original_array)):
                if row_indices[i]: 
                    group.append(row_indices[i].pop(0))
                    flag = True
                if len(group) == 10:  
                    break
            if not flag: 
                break
        if group:
            result.append(group)
        if not any(row_indices.values()):
            break
    
    output_ids = [id for group in result for id in group]
    if len(output_ids) != total_ids:
        raise ValueError("wrong number!")
    
    return result


In [52]:

def process_sampled_ids(csv_file,sample_ids_list):
    execution_time=0
    use_number = 0
    total_tokens_call = 0
    
    all_classified_results = []
    for ids in sample_ids_list:
        content_prompt = get_prompt_from_ids(id_list = ids, file_path = csv_file)
        start_time = time.time()
        completion = client.chat.completions.create(
                            model = "gpt-4o-mini",
                            messages=[
                            {"role": "system", "content": "You are a worker with rich experience performing Entity Resolution tasks. You specialize in clustering and classification within ER."},
                            {"role": "user", "content": pre_prompt + content_prompt},
                            ]
                        )
        execution_time += (time.time() - start_time)
        use_number += 1
        token_number = completion.usage.total_tokens
        total_tokens_call += token_number
        content = completion.choices[0].message.content
        content = content.replace('\n', '').replace(' ', '')
        content_cleaned = re.sub(r"[^\d\[\],]", "", content)
        content_cleaned = re.sub(r",\s*]", "]", content_cleaned)
        content_cleaned = re.sub(r",+", ",", content_cleaned)
        matches = re.findall(r'\[([^\[\]]*?)\]', content_cleaned)
        result_llm = []
        import builtins
        for match in matches:
            match_cleaned = match.strip()
            if ',' in match_cleaned:
                sublist = [int(num) for num in match_cleaned.split(',')]
                result_llm.append(sublist)
            else:
                result_llm.append([int(num) for num in match_cleaned.split()])
        all_classified_results.append(result_llm) 
    return all_classified_results,execution_time,use_number,total_tokens_call
        

In [53]:

def the_most_importent_one(vector_data,classified_results):
    result_for_find = []
    for classified_results_row in classified_results:
        list_select = []
        vectored_select = []
        for cluster_row in classified_results_row:
            vectored_select = np.array([vector_data[id_] for id_ in cluster_row])
            avg_vector = np.mean(vectored_select, axis=0) 
            distances = [np.linalg.norm(vectored_select[i] - avg_vector) for i in range(len(cluster_row))]
            representative_id = cluster_row[np.argmin(distances)]
            list_select.append(representative_id)
        result_for_find.append(list_select)
    return result_for_find


In [54]:
def read_2d_array_from_file(file_path):
    array_list = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                row = list(map(int, line.strip().split()))
                array_list.append(row)
        return array_list
    except FileNotFoundError:
        print(f"{file_path}  is not found !")
    except ValueError as e:
        print(f"Exist wrong number ：{e}")
    return []



In [55]:

def the_most_importent_one_1(classified_results):
    result_for_find = []
    for classified_results_row in classified_results:
        list_select = []
        for cluster_row in classified_results_row:   
            representative_id = cluster_row[0]
            list_select.append(representative_id)
        result_for_find.append(list_select)
    return result_for_find

In [56]:
from typing import List, Optional
def find_most_similar(
    current_id: Optional[int], 
    candidate_ids: List[int], 
    similarity_matrix: List[List[float]]
) -> Optional[int]:

    if not candidate_ids:
        return None
    
    if current_id is None:
        return candidate_ids[0]

    max_similarity = -float('inf')
    most_similar_id = None
    
    for candidate_id in candidate_ids:
        similarity = similarity_matrix[current_id][candidate_id]
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_id = candidate_id
    
    return most_similar_id

def filter_available_ids(next_row_ids: List[int], id_assigned: set) -> List[int]:
    return [id_ for id_ in next_row_ids if id_ not in id_assigned]

def process_rounds(
    id_matrix: List[List[int]],
    similarity_matrix: List[List[float]],
    max_length: int
) -> List[List[int]]:
    all_rounds = []
    id_assigned = set()
    total_rows = len(id_matrix)

    while True:
        current_round = []
        current_id = None

        for row_index in range(total_rows):
            available_ids = filter_available_ids(id_matrix[row_index], id_assigned)
            if not available_ids:
                continue

            next_id = find_most_similar(current_id, available_ids, similarity_matrix)
            if next_id is not None:
                current_round.append(next_id)
                id_assigned.add(next_id)
                current_id = next_id
        if not current_round:
            break
        
        all_rounds.append(current_round)
    
    return all_rounds



def traverse_ids_to_2d(
    id_matrix: List[List[int]], 
    similarity_matrix: List[List[float]],
    max_length: int = 10, 
    batch_size: int = 10
) -> List[List[int]]:
    all_rounds = process_rounds(id_matrix, similarity_matrix, max_length)
    return all_rounds

In [57]:
def merge_find(x, pa):
    if pa[x] != x:
        pa[x] = merge_find(pa[x], pa)  
    return pa[x]


def merge_union(x, y, pa):
    rootX = merge_find(x, pa)
    rootY = merge_find(y, pa)
    if rootX != rootY:
        pa[rootY] = rootX  


def find_simi_nex(small_clusters, now_cluster , pa , ini_simi,the_max_nex):
    global i1, i2
    pattern = [] 
    for i in range(len(now_cluster) - 1):
        for j in range(i + 1, len(now_cluster)):
            if ini_simi[now_cluster[i]][now_cluster[j]] >= the_max_nex:
                # print(maper[now_cluster[i]], maper[now_cluster[j]])
                # print([now_cluster[i], now_cluster[j]])
                pattern.append([now_cluster[i], now_cluster[j]])
    for x, y in pattern:
        for i in range(len(small_clusters)):
            if x in small_clusters[i]:
                i1 = i
                break
        for i in range(len(small_clusters)):
            if y in small_clusters[i]:
                i2 = i
                break
        merge_union(i1, i2, pa)
    merged_groups = {}
    for i in range(len(small_clusters)):
        root = merge_find(i, pa)
        if root not in merged_groups:
            merged_groups[root] = []
        merged_groups[root].extend(small_clusters[i])
    result = [sorted(set(values)) for values in merged_groups.values()]
    return result


In [ ]:
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from itertools import combinations

def is_satisfied(result_matrix, similarity_matrix):
    sim_matrix = np.array(similarity_matrix)
    
    min_intra = float('inf')
    for group in result_matrix:
        if len(group) < 2:
            continue  
        pairs = np.array([(i, j) for i in group for j in group if i < j])
        if len(pairs) == 0:
            continue
        sims = sim_matrix[pairs[:, 0], pairs[:, 1]]
        group_min = np.min(sims)
        if group_min < min_intra:
            min_intra = group_min
    
    max_inter = float('-inf')
    
    group_sets = [set(group) for group in result_matrix]
    group_pairs = list(combinations(range(len(result_matrix)), 2))
    
    with ProcessPoolExecutor() as executor:
        future_results = []
        for i, j in group_pairs:
            future = executor.submit(
                compute_max_inter_similarity, 
                group_sets[i], group_sets[j], sim_matrix
            )
            future_results.append(future)
        
        for future in future_results:
            group_pair_max = future.result()
            if group_pair_max > max_inter:
                max_inter = group_pair_max
    return min_intra >= max_inter

def compute_max_inter_similarity(group_i, group_j, sim_matrix):
    max_sim = float('-inf')
    for i in group_i:
        for j in group_j:
            sim = sim_matrix[i, j]
            if sim > max_sim:
                max_sim = sim
    return max_sim

In [58]:
def check_and_handle_missing_ids(one_slice, result_tmp, attempt):
    detected_ids = set()
    for group in result_tmp:
        detected_ids.update(group)
    missing_ids = set(one_slice) - detected_ids
    
    if not missing_ids:
        return result_tmp, True
    else:
        if attempt == 0:
            print(f"Retrying...")
        else:
            for missing_id in missing_ids:
                result_tmp.append([missing_id])
        return result_tmp, False


def llm_seperate(data_list, data_file, ini_simi, the_max_nex):
    api_call_time = 0
    use_time = 0
    use_token = 0
    seperate_input_token = 0
    seperate_output_token = 0
    result_sliced = []
    number = math.ceil(len(data_list) / 10)
    sliced_lists = [data_list[i * 10:(i + 1) * 10] for i in range(number)]

    for one_slice in sliced_lists:
        api_call_time += 1
        for attempt in range(2):  
            start_time = time.time()
            prompt_sliced = get_prompt_from_ids(one_slice, data_file)
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system",
                     "content": "You are a worker specialize in clustering and classification within Entity Resolution."},
                    {"role": "user", "content": pre_prompt + prompt_sliced},
                ]
            )
            use_time += time.time() - start_time
            prompt_tokens = completion.usage.prompt_tokens  
            seperate_input_token += prompt_tokens
            completion_tokens = completion.usage.completion_tokens 
            seperate_output_token += completion_tokens
            token_number = completion.usage.total_tokens
            use_token += token_number
            content = completion.choices[0].message.content
            content = content.replace('\n', '').replace(' ', '')
            content_cleaned = re.sub(r"[^\d\[\],]", "", content)
            content_cleaned = re.sub(r",\s*]", "]", content_cleaned)
            content_cleaned = re.sub(r",+", ",", content_cleaned)
            matches = re.findall(r'\[([^\[\]]*?)\]', content_cleaned)
            result_tmp = []
            for match in matches:
                match_cleaned = match.strip()
                if ',' in match_cleaned:
                    sublist = [int(num) for num in match_cleaned.split(',')]
                    result_tmp.append(sublist)
                else:
                    result_tmp.append([int(num) for num in match_cleaned.split()])

            result_tmp, complete = check_and_handle_missing_ids(one_slice, result_tmp, attempt)
            if complete:
                break

        for row_slice in result_tmp:
            result_sliced.append(row_slice)
    parent = list(range(len(result_sliced)))
    array_new = find_simi_nex(result_sliced, data_list, parent, ini_simi, the_max_nex)
    return array_new, api_call_time, use_time, use_token, seperate_input_token, seperate_output_token


In [59]:

def find_back(two_d_array, three_d_array):

    num_to_row = {}
    for matrix in three_d_array:
        for row in matrix:
            for number in row:
                if number not in num_to_row:
                    num_to_row[number] = row
    for i, row in enumerate(two_d_array):
        new_row = []
        for number in row:
            if number in num_to_row:
                new_row.extend(num_to_row[number])
        two_d_array[i] = list(dict.fromkeys(new_row))
    return two_d_array

In [60]:

def cal_total_simi_vector(data_file_path,model_file):
    model = SentenceTransformer(model_file)
    def combine_attributes(row):
        return ' '.join(str(value) for value in row[1:]) 
    data = pd.read_csv(data_file_path,encoding="MacRoman")
    data['combined_text'] = data.apply(combine_attributes, axis=1)
    vectors = data['combined_text'].apply(lambda text: model.encode(text)).tolist() 
    simi_matrix = cosine_similarity(vectors)
    print("calculate similarity matrix done")
    return vectors,simi_matrix,data

In [61]:
import networkx as nx
import numpy as np

def bipartite_clustering(data, similarity_matrix):
    G = nx.Graph()
    for i in range(len(similarity_matrix)):
        G.add_node(i)
    for i in range(len(similarity_matrix)):
        for j in range(len(similarity_matrix)):
            if similarity_matrix[i][j] > 0:
                G.add_edge(i, j)
    node_partition = nx.bipartite.color(G)
    cluster1 = []
    cluster2 = []
    for i, color in node_partition.items():
        if color == 0:
            cluster1.append(data[i])
        else:
            cluster2.append(data[i])

    return [cluster1, cluster2]


In [62]:

import csv
from collections import Counter

def read_clusters_from_csv(filename):
    clusters = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            clusters.append([int(item) for item in row if item])  
    return clusters

def calculate_purity(true_clusters, predicted_clusters):
    total_samples = sum(len(cluster) for cluster in predicted_clusters)
    total_correct = 0

    for pred_cluster in predicted_clusters:
        label_count = Counter()
        for sample in pred_cluster:
            for true_cluster in true_clusters:
                if sample in true_cluster:
                    label_count[tuple(true_cluster)] += 1
        if label_count:
            max_label_count = max(label_count.values())
            total_correct += max_label_count

    return total_correct / total_samples if total_samples > 0 else 0

def calculate_inverse_purity(true_clusters, predicted_clusters):
    total_samples = sum(len(cluster) for cluster in true_clusters)
    total_correct = 0

    for true_cluster in true_clusters:
        if true_cluster:
            pred_labels = Counter()
            for sample in true_cluster:
                for pred_cluster in predicted_clusters:
                    if sample in pred_cluster:
                        pred_labels[tuple(pred_cluster)] += 1
            if pred_labels:
                max_match = max(pred_labels.values())
                total_correct += max_match

    return total_correct / total_samples if total_samples > 0 else 0

def calculate_fp_measure(true_clusters, predicted_clusters):
    purity = calculate_purity(true_clusters, predicted_clusters)
    inverse_purity = calculate_inverse_purity(true_clusters, predicted_clusters)

    if purity + inverse_purity == 0:
        return 0

    return 2 * (purity * inverse_purity) / (purity + inverse_purity)
import numpy as np
from scipy.optimize import linear_sum_assignment

def convert_to_labels(clusters, n_samples):
    labels = [-1] * n_samples 
    for cluster_id, cluster in enumerate(clusters):
        for sample in cluster:
            labels[sample] = cluster_id
    return labels

from sklearn.metrics import adjusted_rand_score
def calculate_ari(true_clusters, predicted_clusters):
    all_samples = set(sample for cluster in true_clusters for sample in cluster) | \
    set(sample for cluster in predicted_clusters for sample in cluster)
    n_samples = max(all_samples) + 1 
    true_labels = convert_to_labels(true_clusters, n_samples)
    predicted_labels = convert_to_labels(predicted_clusters, n_samples)
    ari = adjusted_rand_score(true_labels, predicted_labels)
    return ari





In [63]:
def seperate(vectors,simi_matrix,merge_clusters_pre,data_file_path,the_max_nex):
    api_call_time_all = 0
    sperate_time = 0
    sperate_token = 0
    seperate_input = 0
    seperate_output = 0
    sperate_result = []
    for id_list in merge_clusters_pre:
        print(id_list)
        text_data = get_data(id_list, data_file_path)
        vectorized_data = vectorize_data(text_data)
        n_clusters = elbow_method(vectorized_data) 
        labels = kmeans_clustering(vectorized_data, n_clusters)
        clusters_labels = format_output(id_list, labels)
        prompt_id = dynamic_sampling(clusters_labels)
        classified_results, execute_time , use_number , total_tokens = process_sampled_ids(data_file_path, prompt_id)
        sperate_time+=execute_time
        api_call_time_all+=use_number
        sperate_token+=total_tokens
        result_for_found = the_most_importent_one(vectors,classified_results) 
        target_list = traverse_ids_to_2d(result_for_found, simi_matrix, max_length=10, batch_size=10) 
        llm_tmp = []
        for row_slice in target_list:
            print(row_slice)
            array_new,api_call_time,use_time,use_token , seperate_input_token , seperate_output_token = llm_seperate(row_slice,data_file_path,simi_matrix,the_max_nex)
            api_call_time_all +=api_call_time
            sperate_time += use_time
            sperate_token += use_token
            seperate_input += seperate_input_token
            seperate_output += seperate_output_token
            llm_tmp = llm_tmp + array_new
        find_back_matrix = find_back(llm_tmp,classified_results)
        sperate_result += find_back_matrix
    print("seperate done")
    return sperate_result, api_call_time_all ,sperate_time, sperate_token, seperate_input , seperate_output

In [64]:
def seperate_jac(simi_matrix,merge_clusters_pre,data_file_path,the_max_nex):
    api_call_time_all = 0
    sperate_time = 0
    sperate_token = 0
    seperate_input = 0
    seperate_output = 0
    sperate_result = []
    for id_list in merge_clusters_pre:
        print(id_list)
        text_data = get_data(id_list, data_file_path)
        vectorized_data = vectorize_data(text_data)
        n_clusters = elbow_method(vectorized_data) 
        labels = kmeans_clustering(vectorized_data, n_clusters)
        clusters_labels = format_output(id_list, labels)
        prompt_id = dynamic_sampling(clusters_labels)
        classified_results, execute_time , use_number , total_tokens = process_sampled_ids(data_file_path, prompt_id)
        sperate_time+=execute_time
        api_call_time_all+=use_number
        sperate_token+=total_tokens

        result_for_found = the_most_importent_one_1(classified_results) 
        target_list = traverse_ids_to_2d(result_for_found, simi_matrix, max_length=10, batch_size=10) 
        llm_tmp = []
        for row_slice in target_list:
            print(row_slice)
            array_new,api_call_time,use_time,use_token , seperate_input_token , seperate_output_token = llm_seperate(row_slice,data_file_path,simi_matrix,the_max_nex)
            api_call_time_all +=api_call_time
            sperate_time += use_time
            sperate_token += use_token
            seperate_input += seperate_input_token
            seperate_output += seperate_output_token
            llm_tmp = llm_tmp + array_new
        find_back_matrix = find_back(llm_tmp,classified_results)
        sperate_result += find_back_matrix
    print("seperate done")
    return sperate_result, api_call_time_all ,sperate_time, sperate_token, seperate_input , seperate_output

In [65]:
def process_id_list(id_list, vectors, simi_matrix, data_file_path):
    text_data = get_data(id_list, data_file_path)
    vectorized_data = vectorize_data(text_data)
    n_clusters = elbow_method(vectorized_data) 
    # n_clusters = 2
    labels = kmeans_clustering(vectorized_data, n_clusters)
    clusters_labels = format_output(id_list, labels)

    prompt_id = dynamic_sampling(clusters_labels)
    classified_results, execute_time, use_number, total_tokens = process_sampled_ids(data_file_path, prompt_id)

    result_for_found = the_most_importent_one(vectors, classified_results)
    target_list = traverse_ids_to_2d(result_for_found, simi_matrix, max_length=10, batch_size=10)

    llm_tmp = []
    api_call_time_all = use_number
    sperate_time = execute_time
    sperate_token = total_tokens

    for row_slice in target_list:
        array_new, api_call_time, use_time, use_token = llm_seperate(row_slice, data_file_path, simi_matrix)
        api_call_time_all += api_call_time
        sperate_time += use_time
        sperate_token += use_token
        llm_tmp.extend(array_new)

    find_back_matrix = find_back(llm_tmp, classified_results)
    return find_back_matrix, api_call_time_all, sperate_time, sperate_token

def seperate_2(vectors, simi_matrix, merge_clusters_pre, data_file_path):
    sperate_result = []
    api_call_time_all = 0
    sperate_time = 0
    sperate_token = 0
    
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_id_list, id_list, vectors, simi_matrix, data_file_path) for id_list in merge_clusters_pre]

        for future in futures:
            result, api_call_time, sperate_time_each, sperate_token_each = future.result()
            sperate_result.extend(result)
            api_call_time_all += api_call_time
            sperate_time += sperate_time_each
            sperate_token += sperate_token_each

    print("separate done")
    return sperate_result, api_call_time_all, sperate_time, sperate_token


In [66]:
def process_id_list(id_list, simi_matrix, data_file_path,the_threhold):

    llm_tmp = []
    api_call_time_all = 0
    sperate_time = 0
    sperate_token = 0
    seperate_token_input = 0
    seperate_token_output = 0
    array_new, api_call_time, use_time, use_token,A,B = llm_seperate(id_list, data_file_path, simi_matrix, the_threhold)
    api_call_time_all += api_call_time
    sperate_time += use_time
    sperate_token += use_token
    seperate_token_input += A
    seperate_token_output += B
    for row in array_new:
        llm_tmp.append(row)


    return llm_tmp, api_call_time_all, sperate_time, sperate_token,seperate_token_input,seperate_token_output

def seperate_4(simi_matrix, merge_clusters_pre, data_file_path,the_threhold):
    sperate_result = []
    api_call_time_all = 0
    sperate_time = 0
    sperate_token = 0
    seperate_input_token = 0
    seperate_out_token = 0
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_id_list, id_list,  simi_matrix, data_file_path,the_threhold) for id_list in merge_clusters_pre]

        for future in futures:
            result, api_call_time, sperate_time_each, sperate_token_each , input_token,output_token = future.result()
            sperate_result.extend(result)
            api_call_time_all += api_call_time
            sperate_time += sperate_time_each
            sperate_token += sperate_token_each
            seperate_input_token += input_token
            seperate_out_token += output_token
    print("separate done")
    return sperate_result, api_call_time_all, sperate_time, sperate_token,seperate_input_token,seperate_out_token


In [68]:
def lsh_block(vectors,data,similarity_threshold):
    lsh = LSHash(hash_size=15, input_dim=384, num_hashtables=8)
    for ix,vec in enumerate(vectors):
        lsh.index(vec,extra_data=ix)
    graph = defaultdict(set)
    for ix, vec in enumerate(vectors):
        results = lsh.query(vec, num_results=None, distance_func='cosine')
        for res in results:
            if res[0][1] is not None:
                jx = res[0][1]
                if jx != ix:
                    similarity = 1 - res[1]
                    if similarity > similarity_threshold:
                        graph[ix].add(jx)
                        graph[jx].add(ix)
    def find_connected_components(graph):
        visited = set()
        components = []

        def dfs(node, component):
            stack = [node]
            while stack:
                current = stack.pop()
                if current not in visited:
                    visited.add(current)
                    component.append(current)
                    for neighbor in graph[current]:
                        if neighbor not in visited:
                            stack.append(neighbor)

        for node in range(len(data)):
            if node not in visited:
                component = []
                dfs(node, component)
                components.append(component)
        return components
    components = find_connected_components(graph)
    clusters_array = []
    clusters = []
    for component in components:
        valid_indices = [idx for idx in component if idx is not None and 0 <= idx < len(data)]
        cluster = data.iloc[valid_indices, 0].tolist()  
        clusters.append(cluster)
        clusters_array.append(valid_indices)
    print("lsh done")
    return clusters

In [69]:
def pick_elements(list1, list2, n=2):
    combined_length = len(list1) + len(list2)
    
    if combined_length <= n:
        return list1 + list2
    
    # Ensure at least one element from each list
    if len(list1) == 0 or len(list2) == 0:
        raise ValueError("Both lists must contain at least one element")
    result = [random.choice(list1), random.choice(list2)]
    
    remaining_slots = n - len(result)
    combined_list = list1 + list2
    combined_list.remove(result[0])
    combined_list.remove(result[1])
    
    # Randomly choose the remaining elements
    result += random.sample(combined_list, remaining_slots)
    
    return result

In [70]:

def get_most_simi(list1,list2,init_simi):
    max_simi = 0
    record_a = 0
    record_b = 0
    for a in list1:
        for b in list2:
            if init_simi[a][b]>max_simi:
                max_simi = init_simi[a][b]
            else:
                continue

    return max_simi

In [71]:

def is_act(a,b,batch_simi):
    for i in range(a+1,b-1):
        if batch_simi[a][i]==0 and batch_simi[i][b]==0:
                return 1
    return 0

In [72]:


def find_top_cells(similarity_matrix , xiaxian,shagnxian ,shuliang):
    triu_indices = np.triu_indices_from(similarity_matrix, k=1)
    triu_values = similarity_matrix[triu_indices]
    valid_indices = np.where((triu_values >= xiaxian) & (triu_values <=shagnxian))[0]
    selected_indices = valid_indices[:shuliang]
    result = [[int(triu_indices[0][i]), int(triu_indices[1][i])] for i in selected_indices]
    return result


In [73]:


def find_merge_cells(similarity_matrix , xiaxian,shagnxian):

    triu_indices = np.triu_indices_from(similarity_matrix, k=1)
    triu_values = similarity_matrix[triu_indices]
    valid_indices = np.where((triu_values >= xiaxian) & (triu_values <=shagnxian))[0]
    result = [[int(triu_indices[0][i]), int(triu_indices[1][i])] for i in valid_indices]
    return result

In [74]:

def merge_2(clusters,simi_matrix,file_path,block_threshold , merge_threshold):
    pre_prompt_merge = ("Do the records in the following clusters refer to the same entity? i.e., given that the records in each "
              "cluster refer to the same entity, can these clusters or parts of these clusters be merged? If they all "
              "point to one entity, answer 'Yes' And returns a two-dimensional array, each dimension of the array is "
              "the cluster id, indicating which clusters can be clustered together, otherwise just answer 'No' with "
              "no reason.You only need to tell me yes or no!!!\n")
    class UnionFind:
        def __init__(self):
            self.parent = {}

        def find(self, x):
            if self.parent[x] != x:
                self.parent[x] = self.find(self.parent[x])
            return self.parent[x]

        def union(self, x, y):
            rootX = self.find(x)
            rootY = self.find(y)
            if rootX != rootY:
                self.parent[rootY] = rootX

        def add(self, x):
            if x not in self.parent:
                self.parent[x] = x

    def merge_coordinates(coordinates):
        uf = UnionFind()
        ids = set()

        for ltable_id, rtable_id in coordinates:
            uf.add(ltable_id)
            uf.add(rtable_id)
            uf.union(ltable_id, rtable_id)
            ids.add(ltable_id)
            ids.add(rtable_id)

        entity_groups = {}
        for _id in ids:
            root = uf.find(_id)
            if root not in entity_groups:
                entity_groups[root] = []
            entity_groups[root].append(_id)

        result_1 = []
        for root, records in entity_groups.items():
            result_1.append(records)

        return result_1
    api_use_time = 0
    merge_time = 0
    merge_token = 0
    merge_input_token = 0
    merge_output_token = 0
    map_merge = [0]*len(clusters)
    row_merge = len(clusters)
    batch_simi = [[0] * row_merge for _ in range(row_merge)]
    for i in range(row_merge):
        for j in range(i,row_merge):
            batch_simi[i][j] = get_most_simi(clusters[i],clusters[j],simi_matrix)
    similarity_matrix = np.array(batch_simi)
    need_merge = [] 
    for threshold in np.arange(merge_threshold+0.02 , block_threshold, 0.02):
        print(threshold)
        selected_target = find_top_cells(similarity_matrix, threshold-0.02, threshold, 10)
        if len(selected_target) == 0:
            continue
        count_yes = 0
        for row_selected in selected_target:
            list_all = pick_elements(clusters[row_selected[0]],clusters[row_selected[1]])
            prompt = get_prompt_from_ids(list_all, file_path)
            start_time = time.time()
            completion = client.chat.completions.create(
                            model = "gpt-4o-mini",
                            messages=[
                            {"role": "system", "content": "You are a worker with rich experience performing Entity Resolution tasks. You specialize in clustering and classification within ER."},
                            {"role": "user", "content": pre_prompt_merge + prompt},
                            ]
                        )
            merge_time += time.time() - start_time
            api_use_time = api_use_time + 1
            prompt_tokens = completion.usage.prompt_tokens  
            merge_input_token += prompt_tokens
            completion_tokens = completion.usage.completion_tokens  
            merge_output_token += completion_tokens
            token_number = completion.usage.total_tokens
            merge_token += token_number
            answer = completion.choices[0].message.content.lower().strip() 
            if 'yes' in answer:
                count_yes += 1
        if count_yes/len(selected_target) >= 0.2:
            need_merge += find_merge_cells(similarity_matrix , threshold-0.02, threshold)
    for row_in_need_merge in need_merge:
        map_merge[row_in_need_merge[0]] = 1
        map_merge[row_in_need_merge[1]] = 1 
    map_rest = []
    for i in range(len(clusters)):
        if map_merge[i] == 0:

            map_rest.append(clusters[i]) 
    new_result = []
    result_1 = merge_coordinates(need_merge)
    for row in result_1:
        tmp = []
        for ids in row:
            tmp.extend(clusters[ids])
        new_result.append(tmp)
    map_rest += new_result
    print("merge_done")
    return map_rest , api_use_time ,merge_time , merge_token , merge_input_token, merge_output_token

In [75]:
def merge(clusters,simi_matrix,file_path):
    pre_prompt_merge = ("Do the records in the following clusters refer to the same entity? i.e., given that the records in each "
              "cluster refer to the same entity, can these clusters or parts of these clusters be merged? If they all "
              "point to one entity, answer 'Yes' And returns a two-dimensional array, each dimension of the array is "
              "the cluster id, indicating which clusters can be clustered together, otherwise just answer 'No' with "
              "no reason.You only need to tell me yes or no!!!\n")
    class UnionFind:
        def __init__(self):
            self.parent = {}
            self.rank = {}
            self.size = {}

        def find(self, x):
            if x not in self.parent:
                self.parent[x] = x
                self.rank[x] = 0
                self.size[x] = 1
            if self.parent[x] != x:
                self.parent[x] = self.find(self.parent[x])
            return self.parent[x]

        def union(self, x, y):
            rootX = self.find(x)
            rootY = self.find(y)

            if rootX != rootY:
                if self.rank[rootX] > self.rank[rootY]:
                    self.parent[rootY] = rootX
                    self.size[rootX] += self.size[rootY]
                elif self.rank[rootX] < self.rank[rootY]:
                    self.parent[rootX] = rootY
                    self.size[rootY] += self.size[rootX]
                else:
                    self.parent[rootY] = rootX
                    self.rank[rootX] += 1
                    self.size[rootX] += self.size[rootY]

        def get_size(self, x):
            rootX = self.find(x)
            return self.size[rootX]

    map_merge = [0]*len(clusters)
    row_merge = len(clusters)
    batch_simi = [[0] * row_merge for _ in range(row_merge)]
    for i in range(row_merge):
        for j in range(i,row_merge):
            batch_simi[i][j] = get_most_simi(clusters[i],clusters[j],simi_matrix)
    min_simi = 0.5891500074006119 
    new = []
    execution_time=0
    use_number = 0
    total_tokens_call = 0

    while True:
        the_max_simi_batch = 0
        the_first_list = 0
        the_second_list = 0
        for i in range(row_merge):
            for j in range(i,row_merge):
 
                if i==j:
                    continue
                elif batch_simi[i][j]>the_max_simi_batch:
                    the_max_simi_batch = batch_simi[i][j]
                    the_first_list = i
                    the_second_list = j
        if the_max_simi_batch < min_simi:
            break
        if batch_simi[the_first_list][the_second_list] > 0.6:
            batch_simi[the_first_list][the_second_list] = min_simi - 0.01
            continue
        is_ok = is_act(the_first_list,the_second_list,batch_simi)
        print(is_ok)
        if is_ok == 1:
            batch_simi[the_first_list][the_second_list] = 0
        else:
            print(batch_simi[the_first_list][the_second_list])
            list1 = clusters[the_first_list]
            list2 = clusters[the_second_list]
            list_all = pick_elements(list1,list2)
            prompt = get_prompt_from_ids(list_all, file_path)
            start_time = time.time()
            completion = client.chat.completions.create(
                            model = "gpt-4o-mini",
                            messages=[
                            {"role": "system", "content": "You are a worker with rich experience performing Entity Resolution tasks. You specialize in clustering and classification within ER."},
                            {"role": "user", "content": pre_prompt_merge + prompt},
                            ]
                        )
            execution_time += (time.time() - start_time)
            use_number = use_number + 1
            token_number = completion.usage.total_tokens
            total_tokens_call += token_number
            answer = completion.choices[0].message.content.lower().strip() 
            if 'yes' in answer:
                print("yes")
                new.append([the_first_list,the_second_list])
                map_merge[the_first_list] = 1
                map_merge[the_second_list] = 1
                batch_simi[the_first_list][the_second_list] = 0
            else:
                print("no")
                batch_simi[the_first_list][the_second_list] =  min_simi-0.01
                continue
    map_rest = []
    for i in range(len(map_merge)):
        if map_merge[i] == 0:
            map_rest.append(clusters[i])
    if new: 
        with open('./nex_step.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(new)
    else:
        print("No data to write to CSV.")
    matches = pd.read_csv('./nex_step.csv')
    column_names = ['ltable_id', 'rtable_id']
    matches.columns = column_names
    uf = UnionFind()
    ids = set()
    for _,row in matches.iterrows():
        uf.union(row['ltable_id'], row['rtable_id'] )
        ids.add(row['ltable_id'])
        ids.add(row['rtable_id'])
    entity_groups = {}
    for _id in ids:
        root= uf.find(_id)
        if root not in entity_groups:
            entity_groups[root] = []
        entity_groups[root].append(_id)
    result_1 = []
    for root, records in entity_groups.items():
        result_1.append(records) 
    result_all_final = result_1+map_rest 
    print("merge done")
    return result_all_final,use_number,execution_time,total_tokens_call

In [76]:

def preprocess(text):
    text = re.sub(r'(?<=\w)([^\w\s]+)(?=\w)', ' ', text)  
    text = re.sub(r'(?<!\w)[^\w\s]+(?!\w)', '', text) 
    text = re.sub(r'(?<=\w)[^\w\s]+(?!\w)', '', text) 
    text = re.sub(r'(?<!\w)[^\w\s]+(?=\w)', '', text)  
    return text.strip().lower() 

def jaccard_similarity_token(str1, str2):
    if len(str1.strip().split()) == 1 and len(str2.strip().split()) == 1 and ('.' in str1 or '.' in str2):
        try:
            num1 = float(next(iter(str1)))
            num2 = float(next(iter(str2)))
            match_length = sum(1 for a, b in zip(str1, str2) if a == b)
            max_length = max(len(str1), len(str2))
            return match_length / max_length

        except ValueError:
            pass
    tokens1 = set(preprocess(str1).split())  
    tokens2 = set(preprocess(str2).split())
    intersection = tokens1.intersection(tokens2)
    union = tokens1.union(tokens2)

    if len(tokens1) == 0 or len(tokens2) == 0:
        return -1.0

    return len(intersection) / len(union)

def calsimi(row_1, row_2, rows):
    record1 = rows[row_1]
    record2 = rows[row_2]
    attributes = set(record1.keys()) - {'id'}
    total_weighted_similarity = 0.0
    total_weight = 0.0
    weights = {attr: 1.0 for attr in attributes}
    for attr in attributes:
        value1 = str(record1[attr])
        value2 = str(record2[attr])
        if value1 == 'nan' or value2 == 'nan':
            continue
        similarity = jaccard_similarity_token(value1, value2)
        if similarity != -1.0:
            weight = weights[attr]
            total_weighted_similarity += similarity * weight
            total_weight += weight
    if total_weight == 0:
        weighted_avg_similarity = 0.0  
    else:
        weighted_avg_similarity = total_weighted_similarity / total_weight
    return weighted_avg_similarity
def jaccard_simi(num_i,rows):
    ini_simi = [[0.0] * num_i for _ in range(num_i)]
    for i in range(num_i):
        for j in range(num_i):
            ini_simi[i][j] = calsimi(i,j,rows)
    return ini_simi

def jaccard_block(num_i,ini_simi,block_threshold):
    class UnionFind:
        def __init__(self):
            self.parent = {}

        def find(self, x):
            if self.parent[x] != x:
                self.parent[x] = self.find(self.parent[x])
            return self.parent[x]

        def union(self, x, y):
            rootX = self.find(x)
            rootY = self.find(y)
            if rootX != rootY:
                self.parent[rootY] = rootX

        def add(self, x):
            if x not in self.parent:
                self.parent[x] = x

    def merge_coordinates(coordinates):
        uf = UnionFind()
        ids = set()

        for ltable_id, rtable_id in coordinates:
            uf.add(ltable_id)
            uf.add(rtable_id)
            uf.union(ltable_id, rtable_id)
            ids.add(ltable_id)
            ids.add(rtable_id)

        entity_groups = {}
        for _id in ids:
            root = uf.find(_id)
            if root not in entity_groups:
                entity_groups[root] = []
            entity_groups[root].append(_id)

        result_1 = []
        for root, records in entity_groups.items():
            result_1.append(records)
        return result_1
    
    candidate_pair = []
    for i in range(num_i):
        for j in range(i,num_i):
            if ini_simi[i][j] >= block_threshold:
                candidate_pair.append([i, j])
    uf = UnionFind()
    block_result = merge_coordinates(candidate_pair)
    return block_result

use jacccard block

In [ ]:

file_path = './dataset/sigmod/'
data_file_path = file_path+'alaska.csv'
gt_path = file_path+'alaska_gt.csv'
df = pd.read_csv(data_file_path, encoding='MacRoman')
rows = df.to_dict(orient='records')
block_threshold = 0.7


num_of_row, num_of_columns = df.shape
block_time = time.time()
ini_simi = jaccard_simi(num_of_row,rows)
print("done")
clusters_block = jaccard_block(num_of_row,ini_simi,block_threshold)
execute_block_time = time.time() -  block_time
print(f"execute_jaccard_block_time (s) : {execute_block_time}")
print(len(clusters_block))

merge_threshold = 0.5  # the optimal threshold gained by `threshold find function`
merge_clusters_pre, use_number, execution_time, total_tokens_call , merge_input_token , merge_output_token = merge_2(clusters_block,ini_simi,data_file_path,block_threshold=block_threshold,merge_threshold=merge_threshold)
print(f"api_call_time_merge: {use_number}")
print(f"time_merge (s): {execution_time}")
print(f"token_merge : {total_tokens_call}")
print(f"token_merge_input : {merge_input_token}")
print(f"token_merge_output : {merge_output_token}")
print(len(merge_clusters_pre))

import warnings
warnings.filterwarnings('ignore')  
not_done_any = []
merge_done = []
for row_line in merge_clusters_pre: 
    if len(row_line)<=5:
        not_done_any.append(row_line)
    else:
        merge_done.append(row_line)
print(len(not_done_any))
seperate_threshold = 0.7

sperate_result, api_call_time_all ,sperate_time, sperate_token, seperate_input_token , seperate_output_token = seperate_jac(ini_simi,merge_done,data_file_path,seperate_threshold)

for row in not_done_any:
    sperate_result.append(row)
print(f"api_call_time_seperate: {api_call_time_all}")
print(f"time_seperate (s): {sperate_time}")
print(f"token_seperate : {sperate_token}")
print(f"token_seperate_input : {seperate_input_token}")
print(f"token_seperate_output : {seperate_output_token}")


true_clusters = get_ground_truth(gt_path)
predicted_clusters = clusters_block
purity = calculate_purity(true_clusters, predicted_clusters)
inverse_purity = calculate_inverse_purity(true_clusters, predicted_clusters)
fp_measure = calculate_fp_measure(true_clusters, predicted_clusters)
acc = calculate_accuracy(true_clusters, predicted_clusters)
ari = calculate_ari(true_clusters, predicted_clusters)
print(f"FP-Measure: {fp_measure}")
print(f"ACC: {acc}")
print(f"ARI: {ari}")

with open(file_path+"JAC_parameters.txt", "w", encoding="utf-8") as file:
    file.write(f"block_thre:{block_threshold} , merge_thre:{merge_threshold} , sep_thre:{seperate_threshold} \n")
    file.write(f"api_call_time_merge: {use_number}\n")
    file.write(f"time_merge (s): {execution_time}\n")
    file.write(f"token_merge : {total_tokens_call}\n")
    file.write(f"token_merge_input : {merge_input_token}\n")
    file.write(f"token_merge_output : {merge_output_token}\n")
    file.write(f"api_call_time_seperate: {api_call_time_all}\n")
    file.write(f"time_seperate (s): {sperate_time}\n")
    file.write(f"token_seperate : {sperate_token}\n")
    file.write(f"token_seperate_input : {seperate_input_token}\n")
    file.write(f"token_seperate_output : {seperate_output_token}\n")
    file.write(f"FP-Measure: {fp_measure}\n")
    file.write(f"ACC: {acc}\n")
    file.write(f"ARI: {ari}\n")
print("done!")